In [1]:
from aizynthfinder.chem import Molecule
import pandas as pd
import tqdm.notebook as tnote
import numpy as np

In [2]:
import os

In [3]:
import math

In [2]:
filename = "C:/Users/Yassine/Desktop/PSC/PSC_Final/Data/False_reactions/Data_False_Reactions_Recommender.csv"

In [11]:
data = pd.read_csv(filename, delimiter=";")

In [12]:
data

,Unnamed: 0,Reactifs,Products
0,0,C[O:2][C:3](=[O:33])/[CH:4]=[C:5](\[O:10][C:11...,C[O:8][C:6](C(=CC(=O)O)[O:10][c:11]1[c:12]([CH...
1,1,O=C1[C:7]2[CH:8]=[CH:9][C:10]([O:12][CH2:13][C...,O=C(O)c1cc(=O)c2ccc(O[CH2:13][CH2:14][CH2:15][...
2,2,[O:1]=[C:2]1[C:7]2[CH:8]=[CH:9][C:10]([O:12][C...,N#Cc1o[c:6]2[c:7]([c:2](=[O:1])[cH:3]1)[cH:8][...
3,3,[O:1]=[C:2]1[C:7]2[CH:8]=[CH:9][C:10]([O:12][C...,c1c(-c2nnn[nH]2)o[c:6]2[c:7]([c:2]1=[O:1])[cH:...
4,4,[O:1]=[C:2]1[C:7]2[CH:8]=[CH:9][C:10]([O:12][C...,O=C([O-])[c:4]1[cH:3][c:2](=[O:1])[c:7]2[c:6](...
...,...,...,...
261820,261820,[H-].[Al+3].[Li+].[H-].[H-].[H-].C(O[C:10]([CH...,OCC1[CH2:13][CH2:14][CH:15]([CH:18]2[O:19][C:2...
261821,261821,[F:1][CH2:2][CH2:3][CH2:4][Br:5].[C:6]1([P:12]...,[Br-]
261822,261822,[F:1][C:2]1[CH:3]=[C:4](Br)[CH:5]=[C:6]([F:8])...,c1c[c:2]([F:1])[cH:7][c:6]([F:8])c1
261823,261823,[N:1]([CH2:4][CH2:5][CH2:6][Si:7]([O:14][CH2:1...,O=C(NC[CH2:5][CH2:6][Si:7]([O:8][CH2:9][CH3:10...


In [6]:
def create_fingerprint(reactants, products):
    prd_f = Molecule(smiles=products).fingerprint(2)
    rea_f = sum(Molecule(smiles=x).fingerprint(2) for x in reactants.split("."))
    return prd_f, prd_f  - rea_f

In [7]:
def string_nonzero_from_fp(fp):
    s = ""
    l = np.nonzero(fp)[0]
    for k in l:
        if (int(fp[k])!=1):
            s+= str(int(k))+"-"+str(int(fp[k]))
        else:
            s += str(int(k))
        s+= " "
    return s

In [8]:
def string_from_fp(fp):
    s = ""
    for k in fp:
        s += str(int(k))
    return s

In [9]:
def create_data_fingerprint(dataset):
   # progress_bar = tnote.tqdm(total = dataset.shape[0])
    res_data = pd.DataFrame(columns=["Product_Fingerprint", "Reaction_Fingerprint"])
    for i in range(dataset.shape[0]):
    #    progress_bar.update(1)
        smarts = dataset.iloc[i,1], dataset.iloc[i,2]
        prd_f, rxn_f = create_fingerprint(smarts[0],smarts[1])
        prd_f, rxn_f = string_nonzero_from_fp(prd_f), string_nonzero_from_fp(rxn_f)
        res_data.loc[i] = [prd_f, rxn_f]
    return res_data

In [13]:
l = []
p = data.shape[0]//200
print(p)
pb = tnote.tqdm(total = p+1)
for i in range(p+1):
    l.append(create_data_fingerprint(data[i*200:(i+1)*200]))
    l[-1].to_csv(f"False_Recommender_FP/False_Reactions_Recommender_{i}_Fingerprints.csv", index = True)
    pb.update(1)

1309


  0%|          | 0/1310 [00:00<?, ?it/s]

In [14]:
res_data_ = pd.concat(l, ignore_index=True)


In [15]:
res_data_.to_csv("False_Reactions_Recommender_Fingerprints.csv")